Propósito geral do notebook/script
----------------------------------
Este notebook realiza duas análises principais em um projeto de RNA‑seq focado em TNBC:
1) **Processamento: lncRNA** – carrega a matriz de contagens por gene, integra a anotação
do GTF (com lncRNAs), computa métricas (CPM, genes detectados), e gera figuras
descritivas (distribuições por biotipo, frações por amostra, top lncRNAs).
2) **Processamento: WGCNA (atualizado)** – carrega os arquivos exportados do WGCNA no
formato Cytoscape (nodes/edges), estima automaticamente um *cutoff* de peso para manter
~5% das arestas (percentil 95), filtra a rede, produz estatísticas estruturais,
contabiliza interações por biotipo, e anota *hub genes* com termos GO via MyGene.info.

### Comum

In [ ]:
# Usado na normalização
try:
    import rpy2.robjects as ro
    from rpy2.robjects import pandas2ri
    pandas2ri.activate()
    have_rpy2 = True
except Exception:
    have_rpy2 = False

if not have_rpy2:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "rpy2>=3.5.1"])
    import rpy2.robjects as ro
    from rpy2.robjects import pandas2ri
    pandas2ri.activate()

# Instala edgeR se necessário
ro.r('if (!requireNamespace("BiocManager", quietly=TRUE)) install.packages("BiocManager", repos="https://cloud.r-project.org")')
ro.r('if (!requireNamespace("edgeR", quietly=TRUE)) BiocManager::install("edgeR", update=FALSE, ask=FALSE)')

In [ ]:
# Importando bibliotecas
import pandas as pd
from google.colab import drive
import os
import re
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import math
import itertools
from scipy import stats
from statsmodels.stats.multitest import multipletests
from sklearn.decomposition import PCA
from collections import Counter
import requests

In [ ]:
def parse_gtf_attributes(attribute_string, keys_to_extract):
  """Extrai pares chave->valor do campo 'attribute' de um GTF.

  Parâmetros
  ----------
  attribute_string : str
  Campo de atributos do GTF (ex.: 'gene_id "ENSG..."; gene_name "TP53"; ...').
  keys_to_extract : list[str]
  Lista de chaves desejadas (ex.: ['gene_id','gene_name','gene_type']).

  Retorna
  -------
  dict
  Dicionário contendo apenas as chaves pedidas, quando presentes.
  """
  pattern = re.compile(r'(\w+)\s+"([^"]+)"')
  extracted = {}
  for k, v in pattern.findall(str(attribute_string)):
      if k in keys_to_extract:
          extracted[k] = v
  return extracted

def tidy_counts_by_biotype(df, sample_cols, biotype_col='gene_type'):
  """Agrega contagens por amostra × biotipo e calcula fração por amostra.

  Parâmetros
  ----------
  df : pd.DataFrame
  DataFrame com colunas de contagens por amostra e uma coluna de biotipo.
  sample_cols : list[str]
  Lista com os nomes das colunas de amostra.
  biotype_col : str, padrão 'gene_type'
  Nome da coluna com o biotipo do gene (ex.: 'lncRNA', 'protein_coding').

  Retorna
  -------
  pd.DataFrame
  Tabela no formato *long* com colunas: 'sample', biotype_col, 'counts',
  'total_counts' e 'fraction' (fração dentro da amostra).
  """
  melted = df.melt(id_vars=[biotype_col], value_vars=sample_cols,
                    var_name='sample', value_name='counts')
  by = melted.groupby(['sample', biotype_col], as_index=False)['counts'].sum()
  totals = by.groupby('sample', as_index=False)['counts'].sum().rename(columns={'counts':'total_counts'})
  out = by.merge(totals, on='sample', how='left')
  out['fraction'] = np.where(out['total_counts']>0, out['counts']/out['total_counts'], 0.0)
  return out

def make_cpm(counts_df, sample_cols):
  """Calcula CPM (counts per million) por coluna de amostra.

  Parâmetros
  ----------
  counts_df : pd.DataFrame
  Matriz de contagens com uma linha por gene e colunas de amostras.
  sample_cols : list[str]
  Nomes das colunas de amostra.

  Retorna
  -------
  (pd.DataFrame, pd.Series)
  - DataFrame de CPM com mesma forma que `counts_df[sample_cols]`.
  - Série com *library sizes* por amostra, após substituir zeros por NaN.
  """
  lib_sizes = counts_df[sample_cols].sum(axis=0)
  lib_sizes = lib_sizes.replace(0, np.nan)
  cpm = counts_df[sample_cols].div(lib_sizes, axis=1) * 1e6
  return cpm, lib_sizes

def load_gtf_gene_biotype(gtf_file):
  """Lê um GTF e retorna um mapa gene_name → gene_type, priorizando biotipos.


  Estratégia:
  - Lê o GTF e *normaliza* o campo attribute.
  - Aceita tanto `gene_type` quanto `gene_biotype`.
  - Filtra apenas features `gene` e remove duplicatas.
  - Quando um gene_name tem múltiplos biotipos (raro), prioriza: lncRNA > protein_coding > outros.


  Retorna DataFrame com colunas ['gene_name','gene_type'].
  """
  cols = ['seqname','source','feature','start','end','score','strand','frame','attribute']
  g = pd.read_csv(gtf_file, sep='\t', comment='#', header=None, names=cols)

  # extrair atributos do GTF (robusto a gene_type vs gene_biotype)
  desired = ['gene_id','gene_name','gene_type','gene_biotype']
  attr = pd.json_normalize(g['attribute'].apply(lambda s: parse_gtf_attributes(s, desired)))

  # padronizar para a coluna 'gene_type'
  if 'gene_type' not in attr or attr['gene_type'].isna().all():
      attr['gene_type'] = attr.get('gene_biotype')

  g = pd.concat([g.drop(columns=['attribute']), attr], axis=1)

  # manter apenas nível gene
  g = (g[g['feature']=='gene'][['gene_name','gene_type']]
          .dropna()
          .drop_duplicates())

  # resolver nomes com biotipos múltiplos (prioriza lncRNA > protein_coding > outros)
  rank = {'lncRNA': 2, 'protein_coding': 1}
  g['rank'] = g['gene_type'].map(rank).fillna(0)
  g = g.sort_values(['gene_name','rank'], ascending=[True,False]).drop_duplicates('gene_name')

  return g[['gene_name','gene_type']]

def parse_traits_from_sample(sample):
  """Extrai traços/condições do nome de amostra (útil para análise posterior).


  Retorna um dicionário binário com chaves: DEX, BRM014, SIGATA6, VEH, DMSO, H4, H24.
  - Usa *string matching* simples; garanta convenção consistente nos nomes de amostras.
  """
  s = sample.upper()
  traits = dict(DEX=0, BRM014=0, SIGATA6=0, VEH=0, DMSO=0, H4=0, H24=0)
  if 'DEX' in s: traits['DEX']=1
  if 'BRM014' in s: traits['BRM014']=1
  if 'SIGATA6' in s: traits['SIGATA6']=1
  if 'VEH' in s: traits['VEH']=1
  if 'DMSO' in s: traits['DMSO']=1
  if '4H' in s: traits['H4']=1
  if '24H' in s: traits['H24']=1
  return traits

def summarize_detection(mask, label):
  """Resume a detecção de genes para um critério dado por `mask`.

  Parâmetros
  ----------
  mask : pd.Series[bool]
  *Boolean mask* indicando quais linhas/genes são considerados detectados.
  label : str
  Rótulo descritivo do critério (usado em gráficos e tabelas).

  Retorna
  -------
  dict
  Contém: total de genes detectados, e subtotais por biotipo (lncRNA, protein_coding, outros).
  """
  sub = merged[mask]
  out = {
      'label': label,
      'n_genes_detected': int(sub.shape[0]),
      'n_lnc_detected':   int(sub[sub['gene_type']=='lncRNA'].shape[0]),
      'n_pc_detected':    int(sub[sub['gene_type']=='protein_coding'].shape[0]),
      'n_other_detected': int(sub[~sub['gene_type'].isin(['lncRNA','protein_coding'])].shape[0]),
  }
  return out

def fetch_go_terms_mygene(symbol_or_ensg):
  """Consulta termos GO (BP/MF/CC) via MyGene.info para um gene humano.

  Parâmetros
  ----------
  symbol_or_ensg : str
  Símbolo do gene (ex.: 'TP53') ou ENSG (ex.: 'ENSG00000141510').

  Retorna
  -------
  tuple(list, list, list)
  Três listas de termos para GO: BP, MF e CC.

  Comentários
  --------
  - Em caso de múltiplos hits, pega o primeiro.
  """
  base = "https://mygene.info/v3/query"
  q = f"symbol:{symbol_or_ensg}"
  if str(symbol_or_ensg).upper().startswith('ENSG'):
      q = f"ensembl.gene:{symbol_or_ensg}"
  params = {"q": q, "species": "human", "fields": "symbol,name,go.BP.term,go.MF.term,go.CC.term", "size": 1}
  try:
      r = requests.get(base, params=params, timeout=20)
      r.raise_for_status()
      hits = r.json().get('hits', [])
      if not hits:
          return [],[],[]
      hit = hits[0]
      def _collect(branch):
          terms = hit.get('go', {}).get(branch, [])
          if isinstance(terms, dict): terms = [terms]
          return [t.get('term','') for t in terms if isinstance(t, dict) and 'term' in t]
      return _collect('BP'), _collect('MF'), _collect('CC')
  except Exception:
      return [],[],[]

In [ ]:
# ============================
# MONTAGEM DO GOOGLE DRIVE (Colab)
# ============================
drive.mount('/content/drive')

In [ ]:
# ============================
# PARÂMETROS E CAMINHOS DO PROJETO
# ============================
SOURCE_FOLDER = '/content/drive/MyDrive/Saude_lncRNA'
COUNTS_TSV   = f'{SOURCE_FOLDER}/data/processed/salmon.merged.gene_counts.tsv'
CHIPSEQ_COUNTS_TSV   = f'{SOURCE_FOLDER}/data/processed/chip_seq_salmon.merged.transcript_counts.tsv'
GTF_FILE     = f'{SOURCE_FOLDER}/data/processed/Anotacao_com_lncRNA.gtf'
NODE_FILE  = f'{SOURCE_FOLDER}/data/processed/CytoscapeInput-nodes.txt'
EDGE_FILE  = f'{SOURCE_FOLDER}/data/processed/CytoscapeInput-edges.txt'

FIG_DIR      = f'{SOURCE_FOLDER}/figures'
LNCRNA_DIR         = f'{FIG_DIR}/lncrna'
CHIPSEQ_LNCRNA_DIR         = f'{FIG_DIR}/lncrna/chip-seq'
WGCNA_DIR         = f'{FIG_DIR}/wgcna'

# Diretórios de entrada/saída específicos do WGCNA
WGCNA_INDIR   = os.path.dirname(NODE_FILE)
WGCNA_OUTDIR  = WGCNA_DIR
HUBS_CSV = f"{SOURCE_FOLDER}/data/processed/hubs.csv"
MODULE_EXPORT_DIR = f"{WGCNA_DIR}/modules"

# Garante que diretórios de saída existam
os.makedirs(LNCRNA_DIR, exist_ok=True)
os.makedirs(CHIPSEQ_LNCRNA_DIR, exist_ok=True)
os.makedirs(WGCNA_DIR, exist_ok=True)
os.makedirs(MODULE_EXPORT_DIR, exist_ok=True)

# Hiperparâmetros e *thresholds* usados em múltiplas etapas
MIN_COUNT = 10 # número mínimo de *reads* em uma amostra para contar como detectado
MIN_SAMPLES = 1 # número mínimo de amostras atingindo MIN_COUNT

PERCENTILE_THRESHOLD = 0.95 # 95th percentile

# Genes para usar nos hubs/modulos
TARGET_GENES = {'GATA6','GATA6-AS1'}  # ajuste se quisermos destacar outros

### Processamento: lncRNA

In [ ]:
# ==========================================
# BLOCO: Processamento — lncRNA (descrição)
# ==========================================
# Este bloco integra contagens com a anotação, computa CPM, sumariza detecção
# e gera figuras descritivas (A–D) em FIG_DIR/lncrna.

#### lncRNA

In [ ]:
# ---------- 1) Ler counts ----------
gene_df = pd.read_csv(COUNTS_TSV, sep='\t')

# Identificar colunas de amostra
fixed_cols = {'gene_id', 'gene_name'}
sample_cols = [c for c in gene_df.columns if c not in fixed_cols]

# Remover versões do gene_id (ex.: ENSG000001.1 -> ENSG000001)
gene_df['gene_id_base'] = gene_df['gene_id'].astype(str).str.split('.').str[0]

In [ ]:
# ---------- 2) Ler GTF (somente 'gene') e extrair atributos ----------
# Definir nomes de colunas padrão para GTF e ler o arquivo ignorando comentários (#)
gtf_cols = ['seqname','source','feature','start','end','score','strand','frame','attribute']

gtf_raw = pd.read_csv(GTF_FILE, sep='\t', comment='#', header=None, names=gtf_cols)

# Extrair apenas as chaves desejadas do campo attribute
desired_keys = ['gene_id','gene_name','gene_type']
attr = gtf_raw['attribute'].apply(lambda s: parse_gtf_attributes(s, desired_keys))
attr_df = pd.json_normalize(attr)

gtf_df = pd.concat([gtf_raw.drop(columns=['attribute']), attr_df], axis=1)

# Filtrar somente a feature=gene, remove duplicatas e pega colunas essenciais
gtf_genes = (gtf_df[gtf_df['feature']=='gene']
             [['gene_id','gene_name','gene_type']]
             .drop_duplicates())

# Remove versões do gene_id para compatibilizar com a contagem
gtf_genes['gene_id_base'] = gtf_genes['gene_id'].astype(str).str.split('.').str[0]

In [ ]:
# ---------- 3) Merge counts + anotação ----------
# Join por gene_id_base e traz gene_type/gene_name do GTF
merged = (gene_df
          .merge(gtf_genes[['gene_id_base','gene_type','gene_name']],
                 on='gene_id_base', how='left', suffixes=('', '_gtf')))

# Priorizar gene_name do counts quando existir; caso contrário, usar do GTF
merged['gene_name_final'] = merged['gene_name'].fillna(merged['gene_name_gtf'])
merged['gene_type'] = merged['gene_type'].fillna('unknown')

In [ ]:
# ---------- 4) Normalização TMM (edgeR) → logCPM ----------
# Prepara matriz de contagens (genes x amostras) para o R
counts_mat = merged[sample_cols].copy()
counts_mat.index = merged['gene_name_final'].fillna(merged['gene_id']).astype(str)

ro.globalenv['counts_df'] = pandas2ri.py2rpy(counts_mat)
ro.r("""
suppressPackageStartupMessages(library(edgeR))
dge <- DGEList(counts = as.matrix(counts_df))
# TMM
dge <- calcNormFactors(dge, method = "TMM")
# logCPM com prior.count=1 (estável para baixa contagem)
logCPM <- cpm(dge, log = TRUE, prior.count = 1)
""")
logCPM_df = pandas2ri.rpy2py(ro.r('as.data.frame(logCPM)'))
logCPM_df.index.name = 'gene'
logCPM_df.columns = sample_cols

In [ ]:
# Anexa média de logCPM ao merged (para gráficos/ordenações)
merged['mean_logCPM'] = logCPM_df.reindex(counts_mat.index).mean(axis=1).values

In [ ]:
# ---------- 5) Definir "detectados" ----------
# Critério 1 (liberal): pelo menos 1 read em qualquer amostra
merged['total_counts'] = merged[sample_cols].sum(axis=1)
detected_any = merged['total_counts'] > 0

# Critério 2 (stringente): >= MIN_COUNT reads em pelo menos MIN_SAMPLES amostras
detected_stringent = (merged[sample_cols] >= MIN_COUNT).sum(axis=1) >= MIN_SAMPLES

In [ ]:
# ---------- 6) Métricas globais ----------
# Resumos por critério — número de genes detectados por biotipo
summary_any = summarize_detection(detected_any, 'any_read>0')
summary_str = summarize_detection(detected_stringent, f'>={MIN_COUNT} reads in >={MIN_SAMPLES} samples')


# Frações de reads por biotipo (global, somando amostras)
reads_by_biotype = merged.groupby('gene_type')[sample_cols].sum()
global_reads = reads_by_biotype.sum(axis=1).sort_values(ascending=False)
global_total = global_reads.sum()

# Fração total por biotipo (útil para checar o peso de lncRNA vs PC)
global_fracs = (global_reads / global_total).rename('fraction')

In [ ]:
# ---------- 7) CPM clássico (não-normalizado) para compatibilidade com v0 dos gráficos ----------
# CPM médio por gene (média nas colunas de amostra)
cpm_raw, lib_sizes = make_cpm(merged, sample_cols)
merged['mean_cpm'] = cpm_raw.mean(axis=1)

In [ ]:
# ---------- 8) Tabelas auxiliares para gráficos ----------

# (a) Tabela longa para barplot: genes detectados por biotipo × critério
det_tab = pd.DataFrame([summary_any, summary_str])
det_long = det_tab.melt(id_vars=['label'],
                        value_vars=['n_lnc_detected','n_pc_detected','n_other_detected'],
                        var_name='class', value_name='n')
det_long['class'] = det_long['class'].map({'n_lnc_detected':'lncRNA',
                                           'n_pc_detected':'protein_coding',
                                           'n_other_detected':'outros'})
det_long['label'] = det_long['label'].map({'any_read>0':'Qualquer leitura (>0)',
                                          f'>={MIN_COUNT} reads in >={MIN_SAMPLES} samples':
                                          f'Robusto (≥{MIN_COUNT} reads em ≥{MIN_SAMPLES} amostras)'})


# (b) Fração por amostra (stacked 100%)
# Constrói tabela por amostra×biotipo e colapsa biotipos menos comuns em "outros"
tidy = tidy_counts_by_biotype(merged, sample_cols, 'gene_type')

main_biotypes = ['protein_coding','lncRNA']
tidy_plot = tidy.copy()
tidy_plot['biotype'] = np.where(tidy_plot['gene_type'].isin(main_biotypes),
                                tidy_plot['gene_type'], 'outros')
tidy_plot = (tidy_plot.groupby(['sample','biotype'], as_index=False)
             [['counts','total_counts']].sum())
tidy_plot['fraction'] = np.where(tidy_plot['total_counts']>0,
                                 tidy_plot['counts']/tidy_plot['total_counts'], 0.0)

# (c) Distribuição de expressão (log10 CPM) para lncRNA vs protein_coding
expr_dist = (merged[merged['gene_type'].isin(['lncRNA','protein_coding'])]
             [['gene_type','mean_logCPM']].copy())

expr_dist['log10_mean_cpm_norm'] = expr_dist['mean_logCPM']  # já é logCPM


In [ ]:
# ---------- 8) Gráficos ----------
# Configuração estética global do seaborn/matplotlib
sns.set_theme(style='whitegrid', context='talk')

# A) Genes detectados por biotipo (dois critérios)
plt.figure(figsize=(10,6))
ax = sns.barplot(data=det_long, x='label', y='n', hue='class',
                 palette={'protein_coding':'#4c78a8','lncRNA':'#f58518','outros':'#54a24b'})
ax.set_xlabel('')
ax.set_ylabel('Genes detectados')
ax.set_title('Genes detectados por biotipo')
ax.legend(title='Biotipo', frameon=False)

plt.tight_layout()

plt.savefig(f'{LNCRNA_DIR}/A_genes_detectados_biotipo.png', dpi=300)
plt.close()

In [ ]:
# B) Fração de reads por amostra (stacked 100%)
# montar barras empilhadas 100%
samples = tidy_plot['sample'].unique()
biotypes = ['protein_coding','lncRNA','outros']
plot_mat = (tidy_plot.pivot(index='sample', columns='biotype', values='fraction')
            .reindex(columns=biotypes).fillna(0.0))
plot_mat = plot_mat.sort_index()

colors = {'protein_coding':'#4c78a8','lncRNA':'#f58518','outros':'#54a24b'}
plt.figure(figsize=(max(12, 0.5*len(samples)), 6))
bottom = np.zeros(len(plot_mat))
for bt in biotypes:
    vals = plot_mat[bt].values
    plt.bar(plot_mat.index, vals, bottom=bottom, color=colors[bt], label=bt)
    bottom += vals
plt.xticks(rotation=75, ha='right')
plt.ylim(0,1)
plt.ylabel('Fração de reads')
plt.title('Fração de reads por biotipo em cada amostra')
plt.legend(title='Biotipo', frameon=False)
plt.tight_layout()

plt.savefig(f'{LNCRNA_DIR}/B_fracao_reads_por_amostra.png', dpi=300)
plt.close()

In [ ]:
# C) Distribuição de expressão (mean CPM) lncRNA vs protein_coding
plt.figure(figsize=(8,6))

ax = sns.violinplot(data=expr_dist, x='gene_type', y='log10_mean_cpm_norm',
                    palette={'protein_coding':'#4c78a8','lncRNA':'#f58518'}, cut=0)

# Overlay com pontos (amostra de até 2000 genes para não sobrecarregar o gráfico)
sns.stripplot(data=expr_dist.sample(min(2000, expr_dist.shape[0]), random_state=42),
              x='gene_type', y='log10_mean_cpm_norm', color='k', size=1, alpha=0.3, jitter=0.2)
ax.set_xlabel('')
ax.set_ylabel('mean logCPM (TMM)')
ax.set_title('Distribuição de expressão (TMM → logCPM)')
plt.tight_layout()

plt.savefig(f'{LNCRNA_DIR}/C_distribuicao_expressao_log10meanCPM.png', dpi=300)
plt.close()

In [ ]:
# D) Top 20 lncRNAs (por expressão média CPM)
plt.figure(figsize=(8,10))

top_lnc = (merged[merged['gene_type']=='lncRNA']
           .sort_values('mean_logCPM', ascending=False)
           .loc[:, ['gene_id','gene_name_final','mean_logCPM']].head(20))

top_lnc_plot = top_lnc.iloc[::-1]  # inverter para barras horizontais de baixo para cima
labels = top_lnc_plot['gene_name_final'].fillna(top_lnc_plot['gene_id'])
plt.barh(labels, top_lnc_plot['mean_logCPM'], color='#f58518')
plt.xlabel('mean logCPM (TMM)')
plt.title('Top 20 lncRNAs por expressão (TMM → logCPM)')
plt.tight_layout()

plt.savefig(f'{LNCRNA_DIR}/D_top20_lncRNAs_meanCPM.png', dpi=300)
plt.close()

In [ ]:
# ---------- 9) Conjunto de lncRNAs "robustos" para usar no WGCNA ----------
robust_lnc_set = set(merged.loc[(merged['gene_type']=='lncRNA') & detected_stringent, 'gene_name_final'].str.upper())

In [ ]:
# === Slide helper: numbers for "Processamento: lncRNA" ===
# Uses variables already created above:
# - gene_df, merged, sample_cols
# - detected_any (≥1 read in any sample)
# - detected_stringent (≥10 reads in ≥MIN_SAMPLES samples)
# - global_fracs (fractions by gene_type across all reads)

# 1) Universe (rows in count matrix)
universe_n = int(gene_df.shape[0])

# 2) Detected (≥1 read in any sample)
det_total = int(detected_any.sum())
det_lnc   = int(((merged['gene_type'] == 'lncRNA') & detected_any).sum())
det_pc    = int(((merged['gene_type'] == 'protein_coding') & detected_any).sum())

# 3) Robust detected (≥10 reads) — MIN_SAMPLES governs how many samples must have ≥10
rob_total = int(detected_stringent.sum())
rob_lnc   = int(((merged['gene_type'] == 'lncRNA') & detected_stringent).sum())
rob_pc    = int(((merged['gene_type'] == 'protein_coding') & detected_stringent).sum())

# 4) Global read distribution by biotype (protein_coding, lncRNA, others)
biotype_series = merged['gene_type'].where(
    merged['gene_type'].isin(['protein_coding','lncRNA']), other='outros'
)
reads_sum = merged.groupby(biotype_series)[sample_cols].sum().sum(axis=1)
reads_pct = (reads_sum / reads_sum.sum() * 100).reindex(
    ['protein_coding','lncRNA','outros']
).fillna(0).round(1)

# Save a tiny CSV for record/slide
slide_tbl = pd.DataFrame({
    'metric': ['universe_genes','detected_total','detected_lncRNA','detected_protein_coding',
               'robust_total','robust_lncRNA','robust_protein_coding',
               'reads_pct_protein_coding','reads_pct_lncRNA','reads_pct_outros'],
    'value':  [universe_n, det_total, det_lnc, det_pc,
               rob_total, rob_lnc, rob_pc,
               reads_pct['protein_coding'], reads_pct['lncRNA'], reads_pct['outros']]
})
slide_tbl.to_csv(f'{LNCRNA_DIR}/lncrna_slide_numbers.csv', index=False)

# Texto para o slide
print(f"Universo: {universe_n:,} genes na matriz de contagem (GSE261989, BT-549).")
print(f"Detectados (≥1 read em qualquer amostra): {det_total:,} genes")
print(f"  • lncRNA: {det_lnc:,}")
print(f"  • protein_coding: {det_pc:,}")
print(f"Detectados robustos (≥10 reads): {rob_total:,} genes ")
print(f"  • lncRNA: {rob_lnc:,}")
print(f"  • protein_coding: {rob_pc:,}")
print("Distribuição global de reads por biotipo (soma em todas as amostras):")
print(f"  • protein_coding: {reads_pct['protein_coding']:.1f}%")
print(f"  • lncRNA: {reads_pct['lncRNA']:.1f}%")
print(f"  • outros: {reads_pct['outros']:.1f}%")

#### Chip-Seq

In [ ]:
# ---------- 1) Ler counts ----------
gene_df = pd.read_csv(CHIPSEQ_COUNTS_TSV, sep='\t')

# Identificar colunas de amostra
fixed_cols = {'gene_id', 'gene_name'}
sample_cols = [c for c in gene_df.columns if c not in fixed_cols]

# Remover versões do gene_id (ex.: ENSG000001.1 -> ENSG000001)
gene_df['gene_id_base'] = gene_df['gene_id'].astype(str).str.split('.').str[0]

In [ ]:
# ---------- 2) Ler GTF (somente 'gene') e extrair atributos ----------
# Definir nomes de colunas padrão para GTF e ler o arquivo ignorando comentários (#)
gtf_cols = ['seqname','source','feature','start','end','score','strand','frame','attribute']

gtf_raw = pd.read_csv(GTF_FILE, sep='\t', comment='#', header=None, names=gtf_cols)

# Extrair apenas as chaves desejadas do campo attribute
desired_keys = ['gene_id','gene_name','gene_type']
attr = gtf_raw['attribute'].apply(lambda s: parse_gtf_attributes(s, desired_keys))
attr_df = pd.json_normalize(attr)

gtf_df = pd.concat([gtf_raw.drop(columns=['attribute']), attr_df], axis=1)

# Filtrar somente a feature=gene, remove duplicatas e pega colunas essenciais
gtf_genes = (gtf_df[gtf_df['feature']=='gene']
             [['gene_id','gene_name','gene_type']]
             .drop_duplicates())

# Remove versões do gene_id para compatibilizar com a contagem
gtf_genes['gene_id_base'] = gtf_genes['gene_id'].astype(str).str.split('.').str[0]

In [ ]:
# ---------- 3) Merge counts + anotação ----------
# Join por gene_id_base e traz gene_type/gene_name do GTF
merged = (gene_df
          .merge(gtf_genes[['gene_id_base','gene_type','gene_name']],
                 on='gene_id_base', how='left', suffixes=('', '_gtf')))

merged['gene_name_final'] = merged['gene_name']
merged['gene_type'] = merged['gene_type'].fillna('unknown')

In [ ]:
# ---------- 4) Normalização TMM (edgeR) → logCPM (robusta) ----------

# 4.0  Matriz genes x amostras
counts_mat = merged[sample_cols].copy()
counts_mat.index = merged['gene_name_final'].fillna(merged['gene_id']).astype(str)

# 4.1  Saneamento p/ edgeR
# - garantir numeric, remover NA
counts_mat = counts_mat.apply(pd.to_numeric, errors='coerce').fillna(0)

# - agrupar nomes duplicados somando
counts_mat = counts_mat.groupby(counts_mat.index).sum()

# - zerar negativos e garantir inteiros
neg_n = int((counts_mat < 0).sum().sum())
if neg_n > 0:
    print(f"[WARN] {neg_n} valores negativos → definidos como 0 para edgeR.")
counts_mat = counts_mat.clip(lower=0).round().astype(np.int64)

# - remover genes com soma 0
counts_mat = counts_mat.loc[counts_mat.sum(axis=1) > 0]

# - remover amostras com biblioteca 0 (causa do 'median(f75)' NA)
lib_sizes = counts_mat.sum(axis=0)
zero_lib_cols = lib_sizes[lib_sizes == 0].index.tolist()
if zero_lib_cols:
    print(f"[WARN] Removendo {len(zero_lib_cols)} amostra(s) com soma de contagens = 0: {zero_lib_cols}")
    counts_mat = counts_mat.drop(columns=zero_lib_cols)
    # atualiza sample_cols para refletir as colunas mantidas
    sample_cols = [c for c in sample_cols if c not in zero_lib_cols]

# checagem final
assert counts_mat.shape[1] > 0, "Nenhuma amostra com contagens > 0 restou após saneamento."

# 4.2  Enviar para R e normalizar (TMM; fallback TMMwsp se necessário)
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import default_converter

# ---- send counts_mat to R and run edgeR ----
with localconverter(ro.default_converter + pandas2ri.converter):
    ro.globalenv['counts_df'] = counts_mat  # auto py2rpy

ro.r("""
suppressPackageStartupMessages(library(edgeR))
dge <- DGEList(counts = as.matrix(counts_df))
dge <- calcNormFactors(dge, method = "TMM")  # or your robust fallback block
logCPM <- cpm(dge, log = TRUE, prior.count = 1)
""")

# ---- bring results back to Python (no double conversion) ----
with localconverter(ro.default_converter + pandas2ri.converter):
    logCPM_df  = ro.r('as.data.frame(logCPM)')          # -> pandas.DataFrame
    logCPM_cols = list(ro.r('colnames(logCPM)'))        # -> Python list

logCPM_df.index.name = 'gene'
logCPM_df.columns = logCPM_cols

# After you compute `logCPM_df` in R and bring it back to Python:

# 1) mean logCPM per gene (index matches counts_mat.index)
mean_logCPM_by_gene = logCPM_df.mean(axis=1)
mean_logCPM_by_gene.index.name = 'gene_key'  # just a label

# 2) Build the same key in `merged` that you used to index counts_mat
#    (you used: gene_name_final filled by gene_id)
merged['gene_key'] = merged['gene_name_final'].fillna(merged['gene_id']).astype(str)

# 3) Map the per-gene values back to *every* row in `merged`
merged['mean_logCPM'] = merged['gene_key'].map(mean_logCPM_by_gene)

# (Optional) sanity check
covered = merged['mean_logCPM'].notna().mean() * 100
print(f"[INFO] mean_logCPM mapped to merged: {covered:.1f}% rows filled")

# If you no longer need the helper key:
# merged.drop(columns='gene_key', inplace=True)

In [ ]:
# ---------- 5) Definir "detectados" ----------
# Critério 1 (liberal): pelo menos 1 read em qualquer amostra
merged['total_counts'] = merged[sample_cols].sum(axis=1)
detected_any = merged['total_counts'] > 0

# Critério 2 (stringente): >= MIN_COUNT reads em pelo menos MIN_SAMPLES amostras
detected_stringent = (merged[sample_cols] >= MIN_COUNT).sum(axis=1) >= MIN_SAMPLES

In [ ]:
# ---------- 6) Métricas globais ----------
# Resumos por critério — número de genes detectados por biotipo
summary_any = summarize_detection(detected_any, 'any_read>0')
summary_str = summarize_detection(detected_stringent, f'>={MIN_COUNT} reads in >={MIN_SAMPLES} samples')


# Frações de reads por biotipo (global, somando amostras)
reads_by_biotype = merged.groupby('gene_type')[sample_cols].sum()
global_reads = reads_by_biotype.sum(axis=1).sort_values(ascending=False)
global_total = global_reads.sum()

# Fração total por biotipo (útil para checar o peso de lncRNA vs PC)
global_fracs = (global_reads / global_total).rename('fraction')

In [ ]:
# ---------- 7) CPM clássico (não-normalizado) para compatibilidade com v0 dos gráficos ----------
# CPM médio por gene (média nas colunas de amostra)
cpm_raw, lib_sizes = make_cpm(merged, sample_cols)
merged['mean_cpm'] = cpm_raw.mean(axis=1)

In [ ]:
# ---------- 8) Tabelas auxiliares para gráficos ----------

# (a) Tabela longa para barplot: genes detectados por biotipo × critério
det_tab = pd.DataFrame([summary_any, summary_str])
det_long = det_tab.melt(id_vars=['label'],
                        value_vars=['n_lnc_detected','n_pc_detected','n_other_detected'],
                        var_name='class', value_name='n')
det_long['class'] = det_long['class'].map({'n_lnc_detected':'lncRNA',
                                           'n_pc_detected':'protein_coding',
                                           'n_other_detected':'outros'})
det_long['label'] = det_long['label'].map({'any_read>0':'Qualquer leitura (>0)',
                                          f'>={MIN_COUNT} reads in >={MIN_SAMPLES} samples':
                                          f'Robusto (≥{MIN_COUNT} reads em ≥{MIN_SAMPLES} amostras)'})


# (b) Fração por amostra (stacked 100%)
# Constrói tabela por amostra×biotipo e colapsa biotipos menos comuns em "outros"
tidy = tidy_counts_by_biotype(merged, sample_cols, 'gene_type')

main_biotypes = ['protein_coding','lncRNA']
tidy_plot = tidy.copy()
tidy_plot['biotype'] = np.where(tidy_plot['gene_type'].isin(main_biotypes),
                                tidy_plot['gene_type'], 'outros')
tidy_plot = (tidy_plot.groupby(['sample','biotype'], as_index=False)
             [['counts','total_counts']].sum())
tidy_plot['fraction'] = np.where(tidy_plot['total_counts']>0,
                                 tidy_plot['counts']/tidy_plot['total_counts'], 0.0)

# (c) Distribuição de expressão (log10 CPM) para lncRNA vs protein_coding
expr_dist = (merged[merged['gene_type'].isin(['lncRNA','protein_coding'])]
             [['gene_type','mean_logCPM']].copy())

expr_dist['log10_mean_cpm_norm'] = expr_dist['mean_logCPM']  # já é logCPM


In [ ]:
# ---------- 8) Gráficos ----------
# Configuração estética global do seaborn/matplotlib
sns.set_theme(style='whitegrid', context='talk')

# A) Genes detectados por biotipo (dois critérios)
plt.figure(figsize=(10,6))
ax = sns.barplot(data=det_long, x='label', y='n', hue='class',
                 palette={'protein_coding':'#4c78a8','lncRNA':'#f58518','outros':'#54a24b'})
ax.set_xlabel('')
ax.set_ylabel('Genes detectados')
ax.set_title('Genes detectados por biotipo')
ax.legend(title='Biotipo', frameon=False)

plt.tight_layout()

plt.savefig(f'{CHIPSEQ_LNCRNA_DIR}/A_genes_detectados_biotipo.png', dpi=300)
plt.close()

In [ ]:
# B) Fração de reads por amostra (stacked 100%)
# montar barras empilhadas 100%
samples = tidy_plot['sample'].unique()
biotypes = ['protein_coding','lncRNA','outros']
plot_mat = (tidy_plot.pivot(index='sample', columns='biotype', values='fraction')
            .reindex(columns=biotypes).fillna(0.0))
plot_mat = plot_mat.sort_index()

colors = {'protein_coding':'#4c78a8','lncRNA':'#f58518','outros':'#54a24b'}
plt.figure(figsize=(max(12, 0.5*len(samples)), 6))
bottom = np.zeros(len(plot_mat))
for bt in biotypes:
    vals = plot_mat[bt].values
    plt.bar(plot_mat.index, vals, bottom=bottom, color=colors[bt], label=bt)
    bottom += vals
plt.xticks(rotation=75, ha='right')
plt.ylim(0,1)
plt.ylabel('Fração de reads')
plt.title('Fração de reads por biotipo em cada amostra')
plt.legend(title='Biotipo', frameon=False)
plt.tight_layout()

plt.savefig(f'{CHIPSEQ_LNCRNA_DIR}/B_fracao_reads_por_amostra.png', dpi=300)
plt.close()

In [ ]:
# C) Distribuição de expressão (mean CPM) lncRNA vs protein_coding
plt.figure(figsize=(8,6))

ax = sns.violinplot(data=expr_dist, x='gene_type', y='log10_mean_cpm_norm',
                    palette={'protein_coding':'#4c78a8','lncRNA':'#f58518'}, cut=0)

# Overlay com pontos (amostra de até 2000 genes para não sobrecarregar o gráfico)
sns.stripplot(data=expr_dist.sample(min(2000, expr_dist.shape[0]), random_state=42),
              x='gene_type', y='log10_mean_cpm_norm', color='k', size=1, alpha=0.3, jitter=0.2)
ax.set_xlabel('')
ax.set_ylabel('mean logCPM (TMM)')
ax.set_title('Distribuição de expressão (TMM → logCPM)')
plt.tight_layout()

plt.savefig(f'{CHIPSEQ_LNCRNA_DIR}/C_distribuicao_expressao_log10meanCPM.png', dpi=300)
plt.close()

In [ ]:
# D) Top 20 lncRNAs (por expressão média CPM)
plt.figure(figsize=(8,10))

top_lnc = (merged[merged['gene_type']=='lncRNA']
           .sort_values('mean_logCPM', ascending=False)
           .loc[:, ['gene_id','gene_name_final','mean_logCPM']].head(20))

top_lnc_plot = top_lnc.iloc[::-1]  # inverter para barras horizontais de baixo para cima
labels = top_lnc_plot['gene_name_final'].fillna(top_lnc_plot['gene_id'])
plt.barh(labels, top_lnc_plot['mean_logCPM'], color='#f58518')
plt.xlabel('mean logCPM (TMM)')
plt.title('Top 20 lncRNAs por expressão (TMM → logCPM)')
plt.tight_layout()

plt.savefig(f'{CHIPSEQ_LNCRNA_DIR}/D_top20_lncRNAs_meanCPM.png', dpi=300)
plt.close()

In [ ]:
# === Slide helper: numbers for "Processamento: lncRNA" ===
# Uses variables already created above:
# - gene_df, merged, sample_cols
# - detected_any (≥1 read in any sample)
# - detected_stringent (≥10 reads in ≥MIN_SAMPLES samples)
# - global_fracs (fractions by gene_type across all reads)

# 1) Universe (rows in count matrix)
universe_n = int(gene_df.shape[0])

# 2) Detected (≥1 read in any sample)
det_total = int(detected_any.sum())
det_lnc   = int(((merged['gene_type'] == 'lncRNA') & detected_any).sum())
det_pc    = int(((merged['gene_type'] == 'protein_coding') & detected_any).sum())

# 3) Robust detected (≥10 reads) — MIN_SAMPLES governs how many samples must have ≥10
rob_total = int(detected_stringent.sum())
rob_lnc   = int(((merged['gene_type'] == 'lncRNA') & detected_stringent).sum())
rob_pc    = int(((merged['gene_type'] == 'protein_coding') & detected_stringent).sum())

# 4) Global read distribution by biotype (protein_coding, lncRNA, others)
biotype_series = merged['gene_type'].where(
    merged['gene_type'].isin(['protein_coding','lncRNA']), other='outros'
)
reads_sum = merged.groupby(biotype_series)[sample_cols].sum().sum(axis=1)
reads_pct = (reads_sum / reads_sum.sum() * 100).reindex(
    ['protein_coding','lncRNA','outros']
).fillna(0).round(1)

# Save a tiny CSV for record/slide
slide_tbl = pd.DataFrame({
    'metric': ['universe_genes','detected_total','detected_lncRNA','detected_protein_coding',
               'robust_total','robust_lncRNA','robust_protein_coding',
               'reads_pct_protein_coding','reads_pct_lncRNA','reads_pct_outros'],
    'value':  [universe_n, det_total, det_lnc, det_pc,
               rob_total, rob_lnc, rob_pc,
               reads_pct['protein_coding'], reads_pct['lncRNA'], reads_pct['outros']]
})
slide_tbl.to_csv(f'{LNCRNA_DIR}/lncrna_slide_numbers.csv', index=False)

# Pretty print (copy/paste into the slide notes)
print("\n===== Texto sugerido para o slide =====\n")
print(f"Universo: {universe_n:,} genes na matriz de contagem (GSE261989, BT-549).")
print(f"Detectados (≥1 read em qualquer amostra): {det_total:,} genes")
print(f"  • lncRNA: {det_lnc:,}")
print(f"  • protein_coding: {det_pc:,}")
print(f"Detectados robustos (≥10 reads): {rob_total:,} genes ")
print(f"  • lncRNA: {rob_lnc:,}")
print(f"  • protein_coding: {rob_pc:,}")
print("Distribuição global de reads por biotipo (soma em todas as amostras):")
print(f"  • protein_coding: {reads_pct['protein_coding']:.1f}%")
print(f"  • lncRNA: {reads_pct['lncRNA']:.1f}%")
print(f"  • outros: {reads_pct['outros']:.1f}%")

#### Diagrama Venn

In [ ]:
# ---- build robust ChIP set correctly (use gene_id fallback, avoid collapsing) ----
rob_chip = merged.loc[detected_stringent, ['gene_id', 'gene_name_final']].copy()
rob_chip['gene_id_base'] = rob_chip['gene_id'].astype(str).str.split('.').str[0]
# key = gene symbol if present, else Ensembl ID (base)
rob_chip['gene_key'] = np.where(
    rob_chip['gene_name_final'].notna() & (rob_chip['gene_name_final'].astype(str).str.strip() != ''),
    rob_chip['gene_name_final'].astype(str),
    rob_chip['gene_id_base'].astype(str)
)
rob_chip['gene_key_up'] = rob_chip['gene_key'].str.upper()

# Diagnostics
n_rows = rob_chip.shape[0]                                 # total robust rows (should be 131)
n_na_symbol = int(rob_chip['gene_name_final'].isna().sum())
n_unique_by_id = rob_chip['gene_id_base'].nunique()
n_unique_by_key = rob_chip['gene_key_up'].nunique()
dup_names = (rob_chip.groupby('gene_key_up').size().sort_values(ascending=False))
dup_names = dup_names[dup_names > 1]

print(f"[ChIP robust] rows={n_rows}, unique_ids={n_unique_by_id}, "
      f"unique_keys={n_unique_by_key}, na_symbols={n_na_symbol}")
if not dup_names.empty:
    print("[ChIP robust] note: multiple Ensembl IDs share the same symbol; top collisions:")
    print(dup_names.head(10))

# FINAL robust set for Venn (size should now match 131 if that’s your robust count)
chip_robust_set = set(rob_chip['gene_key_up'])

# ---- (re)make the Venn with this corrected set ----
from matplotlib import pyplot as plt
try:
    from matplotlib_venn import venn2, venn2_unweighted
except ImportError:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "matplotlib-venn"])
    from matplotlib_venn import venn2, venn2_unweighted

A = chip_robust_set                  # ChIP robust (should be 131)
B = set(robust_lnc_set)              # RNA-seq lncRNA robust (uppercase)

a_only = len(A - B)
b_only = len(B - A)
ab     = len(A & B)
na, nb = len(A), len(B)

imbalance = min(na, nb) / max(na, nb) if max(na, nb) else 1.0
use_unweighted = (imbalance < 0.4)

plt.figure(figsize=(8,6))
v = (venn2_unweighted if use_unweighted else venn2)(
    subsets=(a_only, b_only, ab),
    set_labels=(f"ChIP-seq robust (n={na:,})", f"RNA-seq lncRNA robust (n={nb:,})")
)

for pid in ('10','01','11'):
    if v.get_patch_by_id(pid):
        v.get_patch_by_id(pid).set_alpha(0.65)
        v.get_patch_by_id(pid).set_edgecolor('#333'); v.get_patch_by_id(pid).set_linewidth(1.2)
if v.get_patch_by_id('10'): v.get_patch_by_id('10').set_color('#4c78a8')
if v.get_patch_by_id('01'): v.get_patch_by_id('01').set_color('#f58518')
if v.get_patch_by_id('11'): v.get_patch_by_id('11').set_color('#9ecae1')

def _fmt(n): return f"{n:,}"
def _pct(n, d): return f"{(n/d*100):.1f}%" if d else "0.0%"

if v.get_label_by_id('10'): v.get_label_by_id('10').set_text(f"{_fmt(a_only)}\n({_pct(a_only, na)})")
if v.get_label_by_id('01'): v.get_label_by_id('01').set_text(f"{_fmt(b_only)}\n({_pct(b_only, nb)})")
if v.get_label_by_id('11'): v.get_label_by_id('11').set_text(f"{_fmt(ab)}")

for lid in ('10','01','11'):
    lbl = v.get_label_by_id(lid)
    if lbl: lbl.set_fontsize(12)
for s in v.set_labels:
    if s: s.set_fontsize(12)

plt.title(f"Sobreposição: ChIP-seq (robusto) × RNA-seq lncRNA (robusto)", fontsize=13, pad=12)
plt.tight_layout()
venn_dir = f"{FIG_DIR}/venn"; os.makedirs(venn_dir, exist_ok=True)
plt.savefig(f"{venn_dir}/venn_chip_vs_rnaseq_lnc_robust_readable.png", dpi=300); plt.close()

# Save the overlap table again (now consistent)
pd.DataFrame(sorted(A & B), columns=['gene']).to_csv(f"{venn_dir}/overlap_genes.csv", index=False)


### Processamento: WGCNA

#### O que esta seção faz
--------------------
1) Lê a anotação (GTF) e constrói um mapa gene_name→biotype.
2) Lê nodes/edges (formato Cytoscape) do WGCNA e anota cada nó com biotipo e módulo.
3) Estima um *cutoff* automático de peso (percentil 95) para manter ~5% das arestas
(apenas para depuração), **e aplica um cutoff calculado (WEIGHT_CUTOFF_AUTO)** na filtragem efetiva.
4) Aplica um filtro biológico adicional: se houver lncRNAs em uma aresta, eles devem
pertencer ao conjunto `robust_lnc_set` (derivado da parte de lncRNA acima; isto
reduz falsos positivos de lncRNAs com baixa/instável detecção).
5) Exporta sub-redes compatíveis com Cytoscape (edges/nodes filtrados) e estatísticas
de redução e estrutura (para slides e QC).
6) Gera *exports por módulo* (nodes/edges) coloridos, marcando hubs e genes-alvo.

In [ ]:
# ---------- 1) Ler GTF e fazer mapa gene_name -> biotipo ----------
# Usa a função utilitária que prioriza lncRNA > protein_coding quando houver ambiguidade
gtf = load_gtf_gene_biotype(GTF_FILE)
name2type = dict(zip(gtf['gene_name'].str.upper(), gtf['gene_type']))

In [ ]:
# ---------- 2) Ler nodes/edges (formato Cytoscape) ----------
nodes = pd.read_csv(NODE_FILE, sep='\t')
if 'nodeAttr[nodesPresent, ]' in nodes.columns:
    nodes = nodes.rename(columns={'nodeAttr[nodesPresent, ]':'module'})
assert {'nodeName','module'}.issubset(nodes.columns), "Arquivo de nodes precisa ter colunas 'nodeName' e 'module'."

# Normalizar nomes e adicionar biotipos
nodes['gene_name']    = nodes['nodeName'].astype(str)
nodes['gene_name_up'] = nodes['gene_name'].str.upper()
nodes['biotype']      = nodes['gene_name_up'].map(name2type).fillna('other')

node2bio    = dict(zip(nodes['nodeName'], nodes['biotype']))
node2module = dict(zip(nodes['nodeName'], nodes['module']))

In [ ]:
# ---------- 3) Encontrar automaticamente o corte que remove 95% das arestas ----------
# **Objetivo:** saber qual peso mantém ~5% das arestas. Isso é útil para calibrar o
# cutoff fixo. A leitura é em *chunks* para economizar memória.

# Histograma de alta resolução para calcular o percentil 95 de 'weight'
hist_bins   = np.linspace(0, 1, 400+1)      # 400 bins p/ boa resolução
hist_counts = np.zeros(len(hist_bins)-1, dtype=np.int64)
total_edges = 0

# Lendo em chunks de 2MI
for ch in pd.read_csv(EDGE_FILE, sep='\t', usecols=['fromNode','toNode','weight'], chunksize=2_000_000):
    ch = ch.dropna()
    w  = ch['weight'].astype(float).values
    h,_ = np.histogram(w, bins=hist_bins)
    hist_counts += h
    total_edges += len(ch)
    del ch
    gc.collect()

# Cálculo do percentil 95 a partir da CDF aproximada do histograma
cum = hist_counts.cumsum() / max(hist_counts.sum(), 1)
idx = np.searchsorted(cum, PERCENTILE_THRESHOLD)
WEIGHT_CUTOFF_AUTO = float(hist_bins[min(idx, len(hist_bins)-1)])

print(f"[INFO] 95th percentile cutoff = {WEIGHT_CUTOFF_AUTO:.6f}  (mantendo ~5% das arestas)")

In [ ]:
# ---------- 4) Filtrar arestas (segunda passada) e salvar o que restou ----------
# Saída: arquivo TSV com arestas filtradas para inspeção no Cytoscape.
filtered_edges_path = f'{WGCNA_DIR}/CytoscapeInput-edges.filtered.top5pct.robust.tsv'
with open(filtered_edges_path, 'w') as f_out:
    f_out.write('fromNode\ttoNode\tweight\n')

# Função auxiliar: garante que lncRNAs envolvidos sejam "robustos"
def _keep_edge(u, v):
    bu, bv = node2bio.get(u, 'other'), node2bio.get(v, 'other')
    if bu == 'lncRNA' and u.upper() not in robust_lnc_set:  # robust_lnc_set vem da parte lncRNA
        return False
    if bv == 'lncRNA' and v.upper() not in robust_lnc_set:
        return False
    return True

nodes_seen = set() # nós que aparecem em pelo menos uma aresta mantida
E_kept = 0

for ch in pd.read_csv(EDGE_FILE, sep='\t', usecols=['fromNode','toNode','weight'], chunksize=2_000_000):
    # 4a) Filtro por peso
    ch = ch[ch['weight'].astype(float) >= WEIGHT_CUTOFF_AUTO].copy()
    if ch.empty:
        continue

    # 4b) Filtro biológico — remove arestas que incluem lncRNA não-robusto
    mask = [ _keep_edge(u, v) for u, v in ch[['fromNode','toNode']].itertuples(index=False, name=None) ]
    ch = ch.loc[mask]
    if ch.empty:
        continue

    # 4c) Export incremental (modo append) para evitar manter tudo em RAM
    ch[['fromNode','toNode','weight']].to_csv(filtered_edges_path, sep='\t', index=False, header=False, mode='a')

    # 4d) Atualiza conjunto de nós presentes
    nodes_seen.update(ch['fromNode'].astype(str))
    nodes_seen.update(ch['toNode'].astype(str))
    E_kept += len(ch)
    del ch

print(f"[INFO] Arestas mantidas após ambos filtros: {E_kept:,} ({E_kept/total_edges*100:.2f}%)")

In [ ]:
# ---------- 5) Salvar nodes filtrados (apenas os presentes nas edges remanescentes) ----------
# Gera um nodes.tsv reduzido (facilita carregar no Cytoscape apenas o subgrafo relevante).

nodes_filt = nodes[nodes['nodeName'].isin(nodes_seen)].copy()
nodes_filt['is_robust_lnc'] = (nodes_filt['biotype'].eq('lncRNA') &
                               nodes_filt['gene_name_up'].isin(robust_lnc_set))

# Removemos lncRNAs não-robustos que, por acaso, tenham aparecido (sanity check)
nodes_filt = nodes_filt[(nodes_filt['biotype']!='lncRNA') | (nodes_filt['is_robust_lnc'])].copy()

nodes_filt_path = f'{WGCNA_DIR}/CytoscapeInput-nodes.filtered.top5pct.robust.tsv'
nodes_filt[['nodeName','gene_name','module','biotype','is_robust_lnc']].to_csv(nodes_filt_path, sep='\t', index=False)

In [ ]:
# ------------------------
# 6) Distribuição das arestas (Original vs Filtrado) p/ o slide
# ------------------------
# Gera uma tabela com contagens de arestas e número de nós por biotipo,
# antes e depois do filtro. Útil para comunicar a magnitude da redução

def _edge_counts(edge_path, filtered=False):
    stats = dict(E_total=0, E_lnc=0, E_gene=0, lnc_nodes=set(), pc_nodes=set())
    node_bio_map = node2bio if not filtered else dict(zip(nodes_filt['nodeName'], nodes_filt['biotype']))
    for ch in pd.read_csv(edge_path, sep='\t', chunksize=2_000_000):
        s_type = ch['fromNode'].map(node_bio_map).fillna('other')
        t_type = ch['toNode'].map(node_bio_map).fillna('other')
        stats['E_total'] += len(ch)
        stats['E_lnc']   += int((s_type.eq('lncRNA') | t_type.eq('lncRNA')).sum())
        stats['E_gene']  += int((s_type.eq('protein_coding') & t_type.eq('protein_coding')).sum())
        stats['lnc_nodes'].update(ch.loc[s_type.eq('lncRNA'),'fromNode'])
        stats['lnc_nodes'].update(ch.loc[t_type.eq('lncRNA'),'toNode'])
        stats['pc_nodes'].update(ch.loc[s_type.eq('protein_coding'),'fromNode'])
        stats['pc_nodes'].update(ch.loc[t_type.eq('protein_coding'),'toNode'])
    return stats

# criar snapshot de edges originais (3 colunas) só para sumarizar
_tmp_all_edges = f'{WGCNA_DIR}/__all_edges_tmp.tsv'
if not os.path.exists(_tmp_all_edges):
    with open(_tmp_all_edges, 'w') as f: f.write('fromNode\ttoNode\tweight\n')
    for ch in pd.read_csv(EDGE_FILE, sep='\t', usecols=['fromNode','toNode','weight'], chunksize=2_000_000):
        ch.to_csv(_tmp_all_edges, sep='\t', index=False, header=False, mode='a')

s_orig = _edge_counts(_tmp_all_edges, filtered=False)
s_filt = _edge_counts(filtered_edges_path, filtered=True)

edges_tbl = pd.DataFrame([
    ('Total de arestas', s_orig['E_total'], s_filt['E_total']),
    ('Nós-lncRNA',       len(s_orig['lnc_nodes']),  len(s_filt['lnc_nodes'])),
    ('Interações lncRNA',s_orig['E_lnc'],          s_filt['E_lnc']),
    ('Nós-Genes',        len(s_orig['pc_nodes']),  len(s_filt['pc_nodes'])),
    ('Interações Genes', s_orig['E_gene'],         s_filt['E_gene']),
], columns=['Métrica','Original','Filtrado'])

edges_tbl['Redução'] = (1 - (edges_tbl['Filtrado']/edges_tbl['Original'])).map(lambda x: f'{x*100:.0f}%')
edges_tbl.to_csv(f'{WGCNA_DIR}/edges_summary.top5pct_robust.csv', index=False)

In [ ]:
# ------------------------
# 7) Estrutura da rede (filtrado) p/ o slide
# ------------------------
# Calculamos grau por nó, densidade, número de componentes e tamanho do maior componente
# usando uma DSU (Union-Find) simples, sem dependências externas.

deg = {}
parent = {}
def _find(x):
    parent.setdefault(x, x)
    while parent[x] != x:
        parent[x] = parent[parent[x]]
        x = parent[x]
    return x
def _union(a,b):
    ra, rb = _find(a), _find(b)
    if ra != rb: parent[rb] = ra

E = 0
for ch in pd.read_csv(filtered_edges_path, sep='\t', chunksize=2_000_000):
    E += len(ch)
    for u,v in ch[['fromNode','toNode']].itertuples(index=False):
        deg[u] = deg.get(u,0)+1; deg[v] = deg.get(v,0)+1
        _union(u,v)

N = len(nodes_filt)
avg_degree = (2*E)/N if N>0 else np.nan
density    = (2*E)/(N*(N-1)) if N>1 else np.nan
comp_size  = {}

for n in nodes_filt['nodeName']:
    r = _find(n); comp_size[r] = comp_size.get(r,0)+1

n_components    = len(comp_size)
giant_component = max(comp_size.values()) if comp_size else 0

net_metrics = pd.DataFrame({
    'Métrica': ['Nós (N)','Arestas (E)','Grau médio (2E/N)','Densidade (2E/N(N-1))','# Componentes','Maior componente (nós)'],
    'Valor':   [N, E, avg_degree, density, n_components, giant_component]
})

net_metrics.to_csv(f'{WGCNA_DIR}/network_structure_metrics.top5pct_robust.csv', index=False)

In [ ]:
# ------------------------
# 8) Módulos — barras coloridas pela cor do módulo
# ------------------------

# Gera uma figura (PNG) com o tamanho de cada módulo e a porcentagem de lncRNA.
WGCNA_COLOR_HEX = {
    'black':'#000000','blue':'#1f77b4','brown':'#8B4513','cyan':'#00FFFF','darkgreen':'#006400',
    'darkgrey':'#555555','darkmagenta':'#8B008B','darkolivegreen':'#556B2F','darkorange':'#FF8C00',
    'darkred':'#8B0000','darkturquoise':'#00CED1','green':'#2ca02c','greenyellow':'#ADFF2F',
    'grey60':'#999999','ivory':'#FFFFF0','lightcyan':'#E0FFFF','lightcyan1':'#E0FFFF',
    'lightgreen':'#90EE90','lightsteelblue1':'#CAE1FF','lightyellow':'#FFFFE0','magenta':'#FF00FF',
    'mediumpurple3':'#8968CD','midnightblue':'#191970','orange':'#FFA500','orangered4':'#8B2500',
    'paleturquoise':'#AFEEEE','pink':'#FFC0CB','plum1':'#FFBBFF','purple':'#800080','royalblue':'#4169E1',
    'saddlebrown':'#8B4513','salmon':'#FA8072','sienna3':'#CD6839','skyblue':'#87CEEB','skyblue3':'#6CA6CD',
    'steelblue':'#4682B4','tan':'#D2B48C','violet':'#EE82EE','white':'#FFFFFF','yellowgreen':'#9ACD32'
}

nodes_mod = nodes_filt.copy()
sizes     = nodes_mod.groupby('module').size().rename('n')
lnc_sizes = nodes_mod[nodes_mod['biotype']=='lncRNA'].groupby('module').size().rename('lnc')
mod_stats = pd.concat([sizes, lnc_sizes], axis=1).fillna(0).reset_index()
mod_stats['lnc_pct'] = (mod_stats['lnc']/mod_stats['n']*100)
mod_stats = mod_stats.sort_values('n', ascending=False)
mod_stats.to_csv(f'{WGCNA_DIR}/modules_sizes_lnc_top5pct_robust.csv', index=False)

plt.figure(figsize=(max(10,0.5*len(mod_stats)),6))
bar_colors = [WGCNA_COLOR_HEX.get(m, '#6baed6') for m in mod_stats['module']]
ax = sns.barplot(data=mod_stats, x='module', y='n', palette=bar_colors)
for i,(m,n,p) in enumerate(zip(mod_stats['module'], mod_stats['n'], mod_stats['lnc_pct'])):
    ax.text(i, n+max(mod_stats['n'])*0.01, f"{p:.1f}% lnc", ha='center', va='bottom', fontsize=10)
plt.xticks(rotation=75, ha='right')
plt.ylabel('Nós no módulo')
plt.xlabel('Módulo (cor)')
plt.title('WGCNA: Módulos — tamanho e % lncRNA (top 5% + lnc robusto)')
plt.tight_layout()

plt.savefig(f'{WGCNA_DIR}/modules_sizes_lnc_colored.png', dpi=300)
plt.close()

In [ ]:
# ------------------------
# 8) Hubs (usar arquivo fornecido, sem API) e marcar nos exports
# ------------------------
# Ler hubs.csv e limpar cores com quebras/aspas
hubs = pd.read_csv(HUBS_CSV, sep=None, engine='python')
hubs.columns = [c.strip().lower() for c in hubs.columns]

# normalizar colunas esperadas: module, gene, function
if 'module' not in hubs.columns or 'gene' not in hubs.columns:
    raise ValueError(f"hubs.csv deve conter colunas 'module' e 'gene'. Encontrado: {list(hubs.columns)}")
if 'function' not in hubs.columns:
    hubs['function'] = ''

hubs['module'] = hubs['module'].astype(str).str.strip().str.replace(r'["\']','', regex=True).str.replace(r'\s+','', regex=True)
hubs['gene']   = hubs['gene'].astype(str).str.strip()
hubs['gene_up'] = hubs['gene'].str.upper()
hubs_map = dict(zip(zip(hubs['module'], hubs['gene_up']), hubs['function']))

In [ ]:
# ------------------------
# 9) Exports por módulo p/ Cytoscape (um par nodes/edges por módulo), destacando hubs e genes-alvo
# ------------------------
TARGET_GENES = {'GATA6','GATA6-AS1'}  # ajuste se quisermos destacar outros
mods = sorted(nodes_mod['module'].unique())
mod_nodesets = {m: set(nodes_mod.loc[nodes_mod['module']==m, 'nodeName']) for m in mods}

# Cabeçalhos — criamos arquivos destinos vazios (modo append virá depois)
for m in mods:
    with open(f'{MODULE_EXPORT_DIR}/{m}_nodes.tsv','w') as fn:
        fn.write('nodeName\tgene_name\tmodule\tbiotype\tis_robust_lnc\tis_hub\thub_function\thighlight\n')
    with open(f'{MODULE_EXPORT_DIR}/{m}_edges.tsv','w') as fe:
        fe.write('fromNode\ttoNode\tweight\n')

# (9a) Nodes por módulo com marcações de hub/lnc robusto/targets
for m in mods:
    sub = nodes_mod[nodes_mod['module']==m].copy()
    sub['is_hub'] = sub['gene_name'].str.upper().apply(lambda g: 1 if (m, g) in hubs_map else 0)
    sub['hub_function'] = sub.apply(lambda r: hubs_map.get((m, r['gene_name'].upper()), ''), axis=1)

    tgt_mask = sub['gene_name'].str.upper().isin(TARGET_GENES)
    lnc_mask = sub['biotype'].eq('lncRNA') & sub['is_robust_lnc'].fillna(True).astype(bool)
    sub['highlight'] = np.select([lnc_mask, tgt_mask], [2, 1], default=0).astype(int)
    sub[['nodeName','gene_name','module','biotype','is_robust_lnc','is_hub','hub_function','highlight']].to_csv(
        f'{MODULE_EXPORT_DIR}/{m}_nodes.tsv', sep='\t', index=False, mode='a', header=False
    )

# (9b) Edges intramodulares por módulo — iteramos os *chunks* das arestas filtradas
for ch in pd.read_csv(filtered_edges_path, sep='\t', chunksize=2_000_000):
    for m in mods:
        s = mod_nodesets[m]
        chm = ch[ch['fromNode'].isin(s) & ch['toNode'].isin(s)]
        if not chm.empty:
            chm.to_csv(f'{MODULE_EXPORT_DIR}/{m}_edges.tsv', sep='\t', index=False, mode='a', header=False)

# (9c) Verificação simples: hubs do CSV estão presentes no subgrafo exportado?
hubs_present = []
for m in mods:
    in_mod = nodes_mod.loc[nodes_mod['module']==m, 'gene_name'].str.upper()
    hl = hubs[hubs['module']==m]
    if not hl.empty:
        gene_up = hl['gene_up'].iloc[0]
        hubs_present.append({
            'module': m,
            'hub_gene': hl['gene'].iloc[0],
            'present_in_network': bool((in_mod==gene_up).any()),
            'function': hl['function'].iloc[0]
        })
pd.DataFrame(hubs_present).to_csv(f'{WGCNA_DIR}/hubs_in_network_check.csv', index=False)

In [ ]:
print(f"[✓] Exports por módulo salvos em: {MODULE_EXPORT_DIR}")
print(f"[✓] Tabelas para slides: edges_summary.top5pct_robust.csv, network_structure_metrics.top5pct_robust.csv, modules_sizes_lnc_colored.png")

In [ ]:
nodes = pd.read_csv(f'{MODULE_EXPORT_DIR}/brown_nodes.tsv', sep="\t")
edges = pd.read_csv(f'{MODULE_EXPORT_DIR}/brown_edges.tsv', sep="\t")

# FIltering lncRNA or GATA6
nodes_lncRNA = nodes[(nodes['biotype'] == 'lncRNA') | (nodes['nodeName'].isin(['GATA6', 'GATA6-AS1']))]

# List of notes
lst = nodes_lncRNA['nodeName'].tolist()

# Filtering only the nodes above
edges = edges[(edges['fromNode'].isin(lst)) | (edges['toNode'].isin(lst))]

# List of nodes on Edges after filtering
lst1 = edges['fromNode'].tolist()
lst2 = edges['toNode'].tolist()

# Filter nodes file with those edges
nodes = nodes[(nodes['nodeName'].isin(lst1)) | (nodes['nodeName'].isin(lst2))]

# Save csv
nodes.to_csv(f'{MODULE_EXPORT_DIR}/brown_nodes_filtered.tsv', sep='\t', index=False)
edges.to_csv(f'{MODULE_EXPORT_DIR}/brown_edges_filtered.tsv', sep='\t', index=False)

### Processamento: DEG

In [ ]:
!apt-get update
!apt-get install -y openjdk-17-jdk
!export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
!export PATH=$PATH:$JAVA_HOME/bin
!wget -qO- https://get.nextflow.io | bash
!chmod +x nextflow
!mv nextflow /usr/local/bin/

In [ ]:
!nextflow run hello

In [ ]:
# ============================
# PARÂMETROS E CAMINHOS DO PROJETO
# ============================
SOURCE_FOLDER = '/content/drive/MyDrive/Saude_lncRNA'
COUNTS_TSV   = f'{SOURCE_FOLDER}/data/processed/salmon.merged.gene_counts.tsv'
CHIPSEQ_COUNTS_TSV   = f'{SOURCE_FOLDER}/data/processed/chip_seq_salmon.merged.transcript_counts.tsv'
GTF_FILE     = f'{SOURCE_FOLDER}/data/processed/Anotacao_com_lncRNA.gtf'
NODE_FILE  = f'{SOURCE_FOLDER}/data/processed/CytoscapeInput-nodes.txt'
EDGE_FILE  = f'{SOURCE_FOLDER}/data/processed/CytoscapeInput-edges.txt'

FIG_DIR      = f'{SOURCE_FOLDER}/figures'
LNCRNA_DIR         = f'{FIG_DIR}/lncrna'
CHIPSEQ_LNCRNA_DIR         = f'{FIG_DIR}/lncrna/chip-seq'
WGCNA_DIR         = f'{FIG_DIR}/wgcna'

# Diretórios de entrada/saída específicos do WGCNA
WGCNA_INDIR   = os.path.dirname(NODE_FILE)
WGCNA_OUTDIR  = WGCNA_DIR
HUBS_CSV = f"{SOURCE_FOLDER}/data/processed/hubs.csv"
MODULE_EXPORT_DIR = f"{WGCNA_DIR}/modules"

# Garante que diretórios de saída existam
os.makedirs(LNCRNA_DIR, exist_ok=True)
os.makedirs(CHIPSEQ_LNCRNA_DIR, exist_ok=True)
os.makedirs(WGCNA_DIR, exist_ok=True)
os.makedirs(MODULE_EXPORT_DIR, exist_ok=True)

# Hiperparâmetros e *thresholds* usados em múltiplas etapas
MIN_COUNT = 10 # número mínimo de *reads* em uma amostra para contar como detectado
MIN_SAMPLES = 1 # número mínimo de amostras atingindo MIN_COUNT

PERCENTILE_THRESHOLD = 0.95 # 95th percentile

# Genes para usar nos hubs/modulos
TARGET_GENES = {'GATA6','GATA6-AS1'}  # ajuste se quisermos destacar outros

# new /content/drive/MyDrive/Saude_lncRNA/data/external and raw
ANNOTATION_GTF_FILE =  f'{SOURCE_FOLDER}/data/external and raw/gencode.v46.annotation.gtf.gz'
LNCRNA_GTF_FILE =  f'{SOURCE_FOLDER}/data/external and raw/gencode.v46.long_noncoding_RNAs.gtf.gz'
GRCH38_GTF_FILE =  f'{SOURCE_FOLDER}/data/external and raw/GRCh38.primary_assembly.genome.fa.gz'


In [ ]:
annotation_gtf = load_gtf_gene_biotype(ANNOTATION_GTF_FILE)
print(annotation_gtf.head())

In [ ]:
lncrna_gtf = load_gtf_gene_biotype(LNCRNA_GTF_FILE)
print(lncrna_gtf.head(300))